Imports

In [1]:
import sys
sys.path.append("../src")

In [2]:
import pandas as pd
from data.wav_data_reader import WavDataReader
from data.signal_dataset import SignalDataset
from features.wav_feature_extractor import WavFeatureExtractor
from features.features_dataset import FeaturesDataset
import data.preparation_eurythmy_data as ped
from models.fully_connected_classifier import FullyConnectedClassifier
from evaluation.hyperparameter_tuner import HyperparameterTuner

Reader

In [3]:
test_folder= r"..\data\interim\testing"

In [4]:
# Initialize the reader with the folder of wavs
reader= WavDataReader(folder= test_folder)

# Get the signals and the keys from the reader
signals, ids= reader.get_values_and_keys()

# Get the measurement labels from the keys of the files
meas_df= ped.return_meas_labels_by_keys(ids)

Signal Dataset

In [5]:
# Initialize SignalDataset with the signals and features/labels
signal_dataset= SignalDataset(signals= signals, features= meas_df)

# Standardize Signals using Zscore
signal_dataset.standardize_signals("zscore")

# Segment Signals in 1sec segments
signal_dataset.segment_signals(segment_duration=1)

# Remove signals whose values are all equal
signal_dataset.remove_constant_signals()

#Add eurythmy letter data to the features
signal_dataset.features= ped.add_meas_letters(signal_dataset.features)

Feature Extractor

In [6]:
extractor= WavFeatureExtractor(sample_rate= 10000, mfccs= True, temporal= True, statistical= True)

Features Dataset

In [7]:
feat_dataset= FeaturesDataset(signal_dataset,extractor)

In [10]:
# Save the dataset
feat_dataset.save_to_csv("raw_dataset.csv")
feat_dataset.save("raw_dataset")
feat_dataset.features.head()

In [ ]:
# Preparation
columns=['duration_seconds', 'flatness_ratio_10000','flatness_ratio_5000', 'flatness_ratio_1000', 'flatness_ratio_500','flatness_ratio_100',]
feat_dataset.drop_columns(columns_to_drop=columns)

## RQ1

Is there any difference in the signals when someone is performing eurythmy?

In [ ]:
# Get data and targets

In [ ]:
train_loader, val_loader, test_loader = feat_dataset.split_dataset_in_loaders(test_size=0.3, val_size=0.5, random_state=42)

## Search

In [ ]:
input_size= num_feat
output_size= 2
num_epochs = 5

param_grid = {
    'learning_rate': [0.1, 0.01],
    'dense_units': [64, 128],
    'dense_layers': [1, 2],
    'dropout_rate': [0.25, 0.5]
}

tuner = HyperparameterTuner(FullyConnectedClassifier, param_grid, train_loader, val_loader, num_epochs, input_size, output_size)
best_params, all_results = tuner.tune()

In [ ]:
print("Best Hyperparameters: \n", best_params)

In [ ]:
all_results.head(15)

In [ ]:
klk

## Model

In [ ]:
num_epochs = 5  # Number of training epochs

In [ ]:
model = FullyConnectedClassifier(
    input_size=5,  # Example input size
    hidden_layers=[128, 64],  # Two hidden layers with 128 and 64 units respectively
    output_size=2,  # Example output size
    dropout_rate=0.5,
    learning_rate=0.001
)

In [ ]:
model.train_model(train_loader, val_loader, num_epochs)

In [ ]:
test_predictions = model.predict(test_loader)

In [ ]:
actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

In [ ]:
correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)
